<a href="https://colab.research.google.com/github/AliHasan-786/Spotify-Music-Recommendation/blob/main/SpotifyMusicRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In order to work on this project you must first:
- Create a Spotify account
- Go to your developer dashboard
- Create a new app

To initiate the construction of a Music Recommendation System, it is essential to first obtain an access token. This access token acts as a temporary authorization credential, enabling the code to perform authenticated requests to the Spotify API on behalf of the application. The following describes the process to acquire this token:

In [2]:
import requests
import base64

CLIENT_ID = 'YOUR_CLIENT_ID'
CLIENT_SECRET = 'YOUR_CLIENT_SECRET'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()



Access token obtained successfully.


In [3]:
import pandas as pd
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']  # Use 'name' instead of 'id'
        album_id = track['album']['id']      # Get the album ID
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        #Get additional audio features (Key, Mode, Speechiness) from the track
        try:
            audio_analysis = sp.audio_analysis(track_id) if track_id != 'Not available' else None
            key = audio_analysis['track']['key'] if audio_analysis else None
            mode = audio_analysis['track']['mode'] if audio_analysis else None
            speechiness = audio_analysis['track']['speechiness'] if audio_analysis else None
        except KeyError:
            key = None
            mode = None
            speechiness = None


        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,


        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 5.3 MB/s eta 0:00:00


In [4]:
playlist_id = '22JbIbnMXs7L8PaF2575av'

#Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

#Display the DataFrame
print(music_df)


                                     Track Name  \
0                                      Location   
1                                    STARGAZING   
2                    rockstar (feat. 21 Savage)   
3                                       Nonstop   
4                                     Freestyle   
..                                          ...   
95                    SKITZO (feat. Young Thug)   
96                       MELTDOWN (feat. Drake)   
97                Jimmy Cooks (feat. 21 Savage)   
98  Knife Talk (with 21 Savage ft. Project Pat)   
99                            Ambition For Cash   

                          Artists            Album Name  \
0                   Playboi Carti         Playboi Carti   
1                    Travis Scott            ASTROWORLD   
2          Post Malone, 21 Savage  beerbongs & bentleys   
3                           Drake              Scorpion   
4                        Lil Baby              Too Hard   
..                            ...

Replace the playlist_id with your own playlist's id




In [5]:
#Check if data has null values or not
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Loudness            0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
Key                 0
Mode                0
Speechiness         0
dtype: int64


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

In [7]:
#Increase the weight for new releases in recommendations

# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [8]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

Create a hybrid recommendation system for music recommendations. The first approach will be based on recommending music based on music audio features, and the second approach will be based on recommending music based on weighted popularity.



In [9]:
#Generate music recommendations based on music audio features

#A function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations = 5):
  if input_song_name not in music_df['Track Name'].values:
    print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
    return

  # Get the index of the input song in the music DataFrame
  input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

  #Calculate the similarity scores based on music features (cosine similarity)
  similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

  #Get the indices of the most similar songs
  similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

  #Get the names of the most similar songs based on content-based filtering
  content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

  return content_based_recommendations

In [10]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations = 5, alpha=0.5):
  if input_song_name not in music_df['Track Name'].values:
    print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
    return

  # Get content-based recommendations
  content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

  #Get the popularity score of the input song
  popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

  #Calculate the weighted popularity score
  weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])


  #Combine content-based and popularity-based recommendations based on weighted popularity
  hybrid_recommendations = content_based_rec
  hybrid_recommendations = hybrid_recommendations.append({
      'Track Name': input_song_name,
      'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
      'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
      'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
      'Popularity': weighted_popularity_score
  }, ignore_index = True)

  #Sort the hybrid recommendations based on weighted popularity score
  hybrid_recommendations = hybrid_recommendations.sort_values(by = 'Popularity', ascending = False)

  #Remove the input song from the recommendations
  hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

  return hybrid_recommendations

In [11]:
#Test the final function to generate music recommendations

input_song_name = "I'm Upset"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'I'm Upset':
                        Track Name                                Artists  \
4               Major Distribution                       Drake, 21 Savage   
0               Both (feat. Drake)                      Gucci Mane, Drake   
1  Commercial (feat. Lil Uzi Vert)                 Lil Baby, Lil Uzi Vert   
2                        Pardon Me  Lil Yachty, Future, Mike WiLL Made-It   
3                          FOR FUN                           Lil Uzi Vert   

                         Album Name Release Date  Popularity  
4                          Her Loss   2022-11-04        74.0  
0  The Return of East Atlanta Santa   2016-12-16        70.0  
1                  My Turn (Deluxe)   2020-05-01        51.0  
2                    Gaming Rap Mix   2021-01-15        22.0  
3                    Hip Hop - Dope   2023-01-06         0.0  


<ipython-input-10-b3583ff7ba85>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hybrid_recommendations = hybrid_recommendations.append({
